This notebook aims to find near duplicates using the SimHash technique, and organizing them in an M-Tree

In [1]:
# import similarities.mtree
import importcsv
import similarities.libsim
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
print('Importing csv file...........')
small_dataset = importcsv.opencsv('processedspeech.csv') 

Importing csv file...........


In [9]:
small_dataset

,member_name,sitting_date,parliamentary_period,parliamentary_session,parliamentary_sitting,political_party,government,member_region,roles,member_gender,speech,processed_speeches
0,κρητικος νικολαου παναγιωτης,03/07/1989,period 5,session 1,sitting 1,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,Παρακαλείται ο κύριος Γραμματέας να συνοδεύσε...,ΠΑΡΑΚΑΛΕΙΤΑ ΚΥΡΙ ΓΡΑΜΜΑΤΕ ΣΥΝΟΔΕΥΣ ΙΕΡ ΣΥΝΟΔ Α...
1,κρητικος νικολαου παναγιωτης,04/07/1989,period 5,session 1,sitting 2,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,"Υπάρχει κανείς εκ των κυρίων συναδέλφων, ο οπ...",ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜ...
2,κρητικος νικολαου παναγιωτης,04/07/1989,period 5,session 1,sitting 2,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,"Κύριοι Βουλευτές, έχω την τιμή να ανακοινώσω ...",ΚΥΡΙ ΒΟΥΛΕΥΤ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΣΩΜ ΒΟΥΛΕΥΤ ΕΒΡ ΥΠΟ...
3,κρητικος νικολαου παναγιωτης,04/07/1989,period 5,session 1,sitting 2,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,Υπάρχει κανείς εκ των κυρίων συναδέλφων που δ...,ΥΠΑΡΧ ΕΚ ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΕΨΗΦΙΣ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩ...
4,κρητικος νικολαου παναγιωτης,04/07/1989,period 5,session 1,sitting 2,πανελληνιο σοσιαλιστικο κινημα,['τζαννετακη τζαννη(02/07/1989-12/10/1989)'],β' πειραιως,['δ αντιπροεδρος βουλης(07/03/1989-21/11/1989)'],male,"Κύριοι συνάδελφοι, έχω την τιμή να ανακοινώσω...",ΚΥΡΙ ΣΥΝΑΔΕΛΦ ΤΙΜ ΑΝΑΚΟΙΝΩΣ ΑΠΟΤΕΛΕΣΜ ΔΙΕΞΑΧΘΕ...
...,...,...,...,...,...,...,...,...,...,...,...,...
362625,βοριδης χρηστου μαυρουδης (μακης),24/07/2020,period 18 review 9,session 1,sitting 187,νεα δημοκρατια,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],α' ανατολικης αττικης,['υπουργος αγροτικης αναπτυξης και τροφιμων(07...,male,Όχι. Θα σας πω. Από πού έχει τεθεί αυτός ο πε...,Π ΤΕΘ ΠΕΡΙΟΡΙΣΜ ΠΑΜ ευρωπαϊκη ΕΝΩΣ ΖΗΤΑΜ ΛΕΜ ...
362626,βοριδης χρηστου μαυρουδης (μακης),24/07/2020,period 18 review 9,session 1,sitting 187,νεα δημοκρατια,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],α' ανατολικης αττικης,['υπουργος αγροτικης αναπτυξης και τροφιμων(07...,male,"Ναι, «δεν υπήρχε».Εμείς είμαστε αναπτυξιακό Υ...",ΥΠΗΡΧΕΕΜ ΑΝΑΠΤΥΞΙΑΚ ΥΠΟΥΡΓΕΙ ΥΠΟΥΡΓΕΙ ΚΑΝ ΚΟΙΝ...
362627,κωνσταντινοπουλος κωνσταντινου οδυσσεας,24/07/2020,period 18 review 9,session 1,sitting 187,κινημα αλλαγης,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],αρκαδιας,['ε αντιπροεδρος βουλης(18/07/2019-28/07/2020)'],male,Ευχαριστούμε τον Υπουργό. Κηρύσσεται περαιωμέ...,ΕΥΧΑΡΙΣΤΟΥΜ ΥΠΟΥΡΓ ΚΗΡΥΣΣΕΤΑ ΠΕΡΑΙΩΜΕΝ ΣΥΖΗΤΗΣ...
362628,κωνσταντινοπουλος κωνσταντινου οδυσσεας,24/07/2020,period 18 review 9,session 1,sitting 187,κινημα αλλαγης,['μητσοτακη κυριακου(08/07/2019-28/07/2020)'],αρκαδιας,['ε αντιπροεδρος βουλης(18/07/2019-28/07/2020)'],male,Μετά την ολοκλήρωση της ψηφοφορίας με το ηλεκ...,ΟΛΟΚΛΗΡΩΣ ΨΗΦΟΦΟΡΙ ΗΛΕΚΤΡΟΝΙΚ ΣΥΣΤΗΜ ΣΧΕΔΙ ΝΟΜ...


Κρατάω μόνο τα processed speeches

In [10]:
speeches = small_dataset['processed_speeches']
# speeches = small_dataset['speech']

In [11]:
print(speeches[0])
print(type(speeches[0]))
# speeches.head()

 Παρακαλείται ο κύριος Γραμματέας να συνοδεύσει την Ιερά Σύνοδο εκτός της Αιθούσης της Βουλής.  . Παρακαλείται ο συνάδελφος Βουλευτής κ. Σαδίκ Αμέτ, που ανήκει στο Μωαμεθανικό Θρήσκευμα να προσέλθει και να δώσει τον οριζόμενο από το Σύνταγμα όρκο επί του Κορανίου.  : ~"Ορκίζομαι στο όνομα του Παντοδύναμου Θεού και του μόνου αυτού Προφήτη ο οποίος είναι ο Μωάμεθ να είμαι πιστός στην πατρίδα και το δημοκρατικό πολίτευμα, να υπακούω στο Σύνταγμα και τους νόμους και να εκπληρώνω ευσυνείδητα τα καθήκοντά μου".
<class 'str'>


We need to define the weights of each word for all speeches, and keep them in a dictionary.
The dictionary is of the form {word: weight}

In [12]:
# This shall come in handy
# https://www.reddit.com/r/LanguageTechnology/comments/dugjis/approximate_string_matching_with_simhash_and_a/

del vectorizer, X
# speeches = speeches.to_list()
vectorizer = TfidfVectorizer(encoding='utf-8', lowercase=False)
X = vectorizer.fit_transform(speeches)

In [13]:
names = vectorizer.get_feature_names_out()
index = 000

print(names[index:index+130])

['00' '000' '0000' '000000' '000000395' '00003' '0001' '00010' '000100'
 '000102' '00010ο' '00011' '000112' '00012' '000123' '00013' '000133'
 '00014' '000144' '000145' '00015' '000155' '00016' '000167' '00017'
 '000178' '00018' '000189' '00019' '0001911' '000192' '0001Α' '0002'
 '00020' '0002013' '00021' '0002116' '00022' '00023' '00024' '00025'
 '000255' '00026' '00027' '00028' '000297' '0002Θεωρούμε' '0002γράφημά'
 '0003' '000319' '0003επιλέξει' '0004' '0004γών' '0005' '0006' '000603'
 '000635' '00067' '0006Α' '0006Β' '0006που' '0007' '000716' '0008' '00081'
 '000858' '0008μοκρατία' '0008ο' '0009' '00091' '000927' '0009Α'
 '0009ορισμένες' '000MW' '000V' '000cc' '000l' '000m3' '000Έτος' '000Αυτά'
 '000Για' '000Διδακτικό' '000Ε' '000Η' '000Ο' '000ΣΕΛΙΔΑ' '000Σύνολο'
 '000Τα' '000Υπερβάλλον' '000Ως' '000άλλοτε' '000β' '000γ' '000δ'
 '000δραχμές' '000δραχμών' '000δρχ' '000ε' '000επί' '000ευρώ' '000ζ'
 '000η' '000και' '000καταστηματάρχες' '000μονάδων' '000νται' '000πάνω'
 '000προηγούμενη